In [1]:
#imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql.types import *
import pandas as pd

In [2]:
#instanciando spark 
spark = SparkSession.builder \
    .master('local[2]') \
    .appName('desafioFInal_EDC') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [3]:
bronze_path = 'D://caiof//Documents//GIT_Repos//EDC-IGTI//desafio_Final//base_dados//datalake//bronze'

In [4]:
##leitura da base de dados
enem_df = (
    spark
    .read
    .format('parquet')
    .option('header',True)
    .option('inferSchema',True)
    .option('encoding','ISO-8859-1')
    .load(bronze_path)
)

enem_df.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

1 - Quantos alunos não quiseram declarar a cor/raça em 2020 (Entenda a opção “não declarado” nessa pergunta)?

In [5]:
raca_n_declarado = (
    enem_df
    .filter(col('TP_COR_RACA') == 0)
    .count()
)

print(raca_n_declarado,'alunos não declararam cor/raça')

116883 alunos não declararam cor/raça


2 - Qual é o número de alunos do Sexo Feminino que estudaram em escola no estado de São Paulo?

In [6]:
f_sp = (
    enem_df
    .filter((col('SG_UF_ESC') == 'SP') & (col('TP_SEXO') == 'F'))
    .count()
)

print(f_sp,'alunas estudaram em uma escola do estado de são paulo')

91480 alunas estudaram em uma escola do estado de são paulo


3 - Quantos alunos do sexo feminino que estudaram em escola no estado do Rio Grande do Sul possuem EXATAMENTE uma geladeira em casa?

In [7]:
#filtrando todas as ALUNAS que estudaram no rio grande do sul 
f_rs = (
    enem_df
    .filter((col('SG_UF_ESC') == 'RS') & (col('TP_SEXO') == 'F'))
)
f_rs.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [8]:
#filtrando os alunos que possuem uma geladeira na residencia

geladeira_1 = (
    f_rs
    .filter(col('Q012') == 'B')
    .count()
)

print(geladeira_1, 'alunas da rede estadual de ensino do rio grande do sul \n possuem exatamento uma geladeira em casa')

18680 alunas da rede estadual de ensino do rio grande do sul 
 possuem exatamento uma geladeira em casa


4 - Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) em que estudaram mais alunos no ENEM 2020?

R: CE

In [9]:
(
    enem_df
    .groupBy('SG_UF_ESC')
    .count()
    .sort(col('count').desc())
    .show(5,truncate=False)
)


+---------+-------+
|SG_UF_ESC|count  |
+---------+-------+
|null     |4878540|
|SP       |167183 |
|CE       |104981 |
|MG       |71614  |
|RJ       |62583  |
+---------+-------+
only showing top 5 rows



5 - Qual é o estado brasileiro (considere a coluna SG_UF_ESC) possui o menor número de alunos cuja mãe possui ensino superior completo?
R: RR


In [10]:
#filtrando todos os alunos que a mãe possui ensino superior
conditions = ['F','G']
mae_superior = (
    enem_df
    .filter(col('Q002').isin(conditions))
    .sort(col('SG_UF_ESC'))
)

mae_superior.show()


+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_A

In [11]:
(
    mae_superior
    .groupBy('SG_UF_ESC')
    .count()
    .sort(col('count').asc())
    .show()
)

+---------+-----+
|SG_UF_ESC|count|
+---------+-----+
|       RR|  795|
|       AP|  930|
|       AC|  932|
|       SE| 1824|
|       TO| 2013|
|       RO| 2071|
|       AL| 2553|
|       PI| 3195|
|       RN| 3225|
|       MS| 3583|
|       PB| 3750|
|       AM| 3989|
|       MA| 4442|
|       MT| 4460|
|       ES| 4556|
|       PA| 6061|
|       DF| 7563|
|       CE| 7738|
|       SC| 7876|
|       GO| 7966|
+---------+-----+
only showing top 20 rows



6 - Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de pessoas na faixa “entre 26 e 30 anos”?
R: AM

In [12]:
m_age = (
    enem_df
    .filter(col('TP_FAIXA_ETARIA') == 11)
)

m_age.show()


+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_AN

In [13]:
(
    m_age
    .groupBy('SG_UF_ESC')
    .count()
    .sort(col('count').desc())
    .show()
)

+---------+------+
|SG_UF_ESC| count|
+---------+------+
|     null|567071|
|       CE|  1880|
|       AM|   828|
|       SP|   580|
|       GO|   518|
|       PA|   479|
|       PE|   461|
|       RJ|   456|
|       BA|   450|
|       MG|   432|
|       PR|   221|
|       RO|   215|
|       PB|   201|
|       RS|   195|
|       SE|   183|
|       MA|   160|
|       PI|   154|
|       TO|   150|
|       RN|   121|
|       ES|   117|
+---------+------+
only showing top 20 rows



7 - Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o TERCEIRO maior número de alunos cuja residência possui PELO MENOS 2 banheiros?
R: RJ

In [14]:
#filtrando todos os registros em que o aluno possui pelo menos 2 banheiros
conditions  = ['C','D','E']

banheiros = (
        enem_df
        .filter(col('Q008').isin(conditions))
)

In [15]:
(
    banheiros
    .groupBy('SG_UF_ESC')
    .count()
    .sort(col('count').desc())
    .show()
)

+---------+-------+
|SG_UF_ESC|  count|
+---------+-------+
|     null|1151573|
|       SP|  83502|
|       MG|  30912|
|       RJ|  24367|
|       PR|  20938|
|       CE|  15533|
|       GO|  14562|
|       RS|  14470|
|       SC|  13104|
|       BA|  13057|
|       PE|  11518|
|       DF|  10231|
|       PA|   8055|
|       ES|   7185|
|       PB|   5978|
|       AM|   5843|
|       MT|   5826|
|       RN|   5439|
|       MA|   5373|
|       MS|   4995|
+---------+-------+
only showing top 20 rows



8 - Quantos alunos do sexo feminino se autodeclararam pretos?

In [16]:
enem_df.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

In [17]:
f_preto = (
    enem_df
    .filter((col('TP_SEXO') == 'F') & (col('TP_COR_RACA') == 2) )
    .count()
)

print(f_preto,'alunas se declararam pretas')

444346 alunas se declararam pretas


9 - Quantos alunos estrangeiros fizeram o ENEM 2020?

In [18]:
#TP_NACIONALIDADE = 3

estrang = (
    enem_df
    .filter(col('TP_NACIONALIDADE') == 3)
    .count()
)

print(estrang,'são alunos estrangeiros')

8036 são alunos estrangeiros


10 - Qual é a diferença da nota média em matemática dos alunos que estudaram o ensino médio em escola pública e em escola privada?

In [19]:
"""
agrupando os alunos por tipo de escola 'privada' 'ou publica'
depois tiro a media da nota de matematica

"""

media_notas = (
    enem_df
    .groupBy('TP_ESCOLA')
    .agg(
        mean('NU_NOTA_MT').alias('media_matematica')
    )
)

In [20]:
"""
Neste step seleciono os valores desejados e transformo em um dataframe pandas
para poder calcular a diferença na media das notas 
"""
media_publica = media_notas.filter(col('TP_ESCOLA') == 2).select('media_matematica').toPandas()['media_matematica']
media_privada = media_notas.filter(col('TP_ESCOLA') == 3).select('media_matematica').toPandas()['media_matematica']
diferenca_media = (media_privada - media_publica).tolist()


In [21]:

print(f'Media alunos escola Privada:{media_privada[0]}\nMedia alunos escola Publica:{media_publica[0]}\ndiferença nota media: {diferenca_media[0]}')

Media alunos escola Privada:610.6393569797274
Media alunos escola Publica:499.52498038446714
diferença nota media: 111.11437659526024


11 - Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de alunos do sexo feminino indígenas?

In [22]:
(
    enem_df
    .filter((col('TP_SEXO') == 'F') & (col('TP_COR_RACA') == 5))
    .groupBy('SG_UF_ESC')
    .count()
    .sort(col('count').desc())
    .show(5)

)

+---------+-----+
|SG_UF_ESC|count|
+---------+-----+
|     null|18081|
|       AM|  398|
|       CE|  380|
|       SP|  298|
|       PE|  249|
+---------+-----+
only showing top 5 rows



12 - Qual é a diferença entre o número de alunos cujo pai possui pós-graduação completa e o número de alunos cuja mãe possui pós-graduação completa?

In [23]:
pai_pos = (
    enem_df
    .filter(col('Q001') == 'G')
    .count()
)

mae_pos = (
    enem_df
    .filter(col('Q002') == 'G')
    .count()
)

In [24]:
print(f'Total de pais pos-graduados: {pai_pos}\nTotal de maes pos-graduadas: {mae_pos}\ndiferenca: {(mae_pos - pai_pos)}')

Total de pais pos-graduados: 239720
Total de maes pos-graduadas: 434437
diferenca: 194717


13 - Quantos alunos cuja residência possui ATÉ 2 carros estudaram na região NORDESTE do Brasil?

In [31]:
conditions = ['MA','PI','CE','RN','PB','PE','AL','SE','BA']
alunos_nordeste = (
    enem_df
    .filter(col('SG_UF_ESC').isin(conditions))
)

alunos_nordeste.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_AN

In [34]:
conditions = ['B','C']
carro = (
    alunos_nordeste
    .filter(col('Q010').isin(conditions))
    .count()
)

print(carro,'Alunos nordestinos que possuem até 2 carros em casa')


74533 Alunos nordestinos possuem até 2 carros em casa


14 - Quantos alunos que estudaram em escolas em zona rural possuem internet em casa?

In [37]:
alunos_rural = (
    enem_df
    .filter(col('TP_LOCALIZACAO_ESC') == 2)
)

alunos_rural.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [39]:
rural_internet = (
    alunos_rural
    .filter(col('Q025') == 'B')
    .count()
)

print(rural_internet,'alunos de região rural possuem internet em casa')

18979 alunos de região rural possuem internet em casa


15 - Qual é o nome do município que contém a SEGUNDA maior quantidade total de inscritos no ENEM 2020?

In [40]:
(
    enem_df
    .groupBy('NO_MUNICIPIO_ESC')
    .count()
    .sort(col('count').desc())
    .show()
)

+----------------+-------+
|NO_MUNICIPIO_ESC|  count|
+----------------+-------+
|            null|4878540|
|       São Paulo|  38277|
|       Fortaleza|  27783|
|  Rio de Janeiro|  26431|
|        Brasília|  19840|
|          Manaus|  17120|
|  Belo Horizonte|  11826|
|          Recife|  10735|
|        Salvador|  10500|
|        Curitiba|   9868|
|           Belém|   9137|
|         Goiânia|   8920|
|        São Luís|   6573|
|        Teresina|   5776|
|    Porto Alegre|   5650|
|          Maceió|   5070|
|     João Pessoa|   5005|
|    Campo Grande|   4767|
|       Guarulhos|   4689|
|           Natal|   4519|
+----------------+-------+
only showing top 20 rows

